 URL은 쿠팡에서 "노트북" 키워드로 검색된 상품 목록에 대한 주소이다.
이 주소의 웹 페이지로부터 상품의 이름, 가격, 할인율, 할인가, 별점, 리뷰수, 상품이미지의 경로를 수집하여 엑셀 파일로 저장하시오.

In [1]:
! pip install --upgrade beautifulsoup4

In [2]:
# 웹브라우저 ->(request)-> 웹서버
# 웹서버 -> (contents) -> 웹브라우저
# contents : json인경우, dictionary변환 / html+css인 경우, beautifulsoup 내용추출
import requests
from bs4 import BeautifulSoup
from pandas import DataFrame

#모듈이 모여있는 것 = 라이브러리
#라이브러리를 배포를 목적으로 압축한 것 = 패키지
#모듈 < 라이브러리 < 패키지

모든 컴퓨터에 부여되는 인터넷상의 주소 = ip address
pc에 naver.com url 입력시 -> DNS에 전송된다. (DNS는 IP와 도메인주소 매칭 데이터를 다룬다.) -> 매칭되는 IP주소를 다시 pc에 전송.

In [3]:
#수집할 컨텐츠가 있는 웹페이지의 주소 ("~? :실제 주소" + "?~":query string )
# query string : 변수이름=값 & 변수이름=값... (한글은 x -> url encoding)
# https:// :프로토콜(통신방식에 대한 규약,규격) (일반웹 : http:// <-> 보안웹: https://)
# www.coupang.com : 도메인 = 컴퓨터 주소
# np/search : 파일 경로

#1) 수집할 url 주소
url = "https://www.coupang.com/np/search?q=%EB%85%B8%ED%8A%B8%EB%B6%81&listSize=72&channel=user"

#2) 브라우저 버전정보
# if 접속객체 없이 request.get(url주소) 요청시, 해당 url에서는 정보를 주지 않음.
# 대부분의 url에서는 접속한 디바이스를 판단. = UserAgent
UserAgent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

#3) 접속객체 생성 및 부가정보 삽입하기
# useragent를 request객체에 실어 보내려면 접속객체를 만들어야 함 = session
session = requests.Session()
session.headers.update({
    "Referer" : "",
    "User-Agent" : UserAgent,
    "Accept-Language": "ko-KR,ko;q=0.8,en-US;q=0.5,en;q=0.3"
})

#4) 생성한 접속객체를 활용하여 API에 접속 & 접속 실패시 오류메세지 출력
r = requests.get(url)
if r.status_code != 200 :
    msg = "[%d Error] %s 에러가 발생함"  %(r.status_code, r.reason)
    raise Exception(msg)

#5) 인코딩 형식 지정하여 beautifulsoup 객체를 생성
# beautifulsoup = 원하는 데이터를 추출하고 객체로
r.encoding = "uft-8"
soup = BeautifulSoup(r.text)
soup

In [ ]:
# 가격(base-value), 할인율(instant-discount-rate), 할인가(price-value), 별점(rating), 리뷰수(others-info), 상품이미지(search-product-wrap)#상품의 이름(name), 
#클래스(class = ".")로 추출

#6) 추출할 데이터를 저장할 빈 리스트 만들기
result = []

#7) .search-product-wrap 클래스만 select => product만 추출
# select의 검색결과는 항상 리스트 형식.
product = soup.select(".search-product-wrap")
product

In [ ]:
#8) product에서 원하는 데이터만 추출 (클래스형식 이용)
for p in product:
    nameEl = p.select(".name")
    if nameEl:
        name = name[0].text.strip()
    else:
        name = None

    basePriceEl = p.select(".base-price")
    if basePriceEl:
        basePrice = int(basePriceEl[0].text.strip().replace(",",""))
    else:
        basePrice = None

    discountEl = p.select(".instant-discount-rate")
    if discountEl:
        discount = int(discountEl[0].text.strip().replace("%",""))/100
    else:
        price = None

    ratingEl = p.select(".rating")
    if ratingEl:
        rating = float(ratingEl[0].text.strip())
    else:
        rating = None
    
    reviewEl = p.select(".rating-total-count")
    if reviewEl:
        review = reviewEl[0].text.strip()
        review = int(review[1:-1])
    else:
        review = None
    
    imageEl = p.select(".search-product-wrap-img")
    if imageEl:
        if "data-img-scr" in imageEl[0].attrs:
            image = "https:" + imageEl[0].attrs['data-img-src']
        else:
            image = "https:" + imageEl[0].attrs['src'] 
    else:
        image = None

    if not basePrice and not price:
        continue

    item = {
        "상품명" : name,
        "가격" : basePrice,
        "할인율" : discount,
        "할인가" : price,
        "별점" : rating,
        "리뷰수" : review,
        "상품이미지" : image
    }

#9) 빈 리스트에 추출한 데이터를 append + dataframe으로 표시
    result.append(item)
    df = DataFrame(result)
    df

 객체지향 언어 
 객체를 구성하는 것은 데이터(변수)와 기능(함수) -> 객체.메서드() 또는 객체.변수

 * 객체(object)
 - 프로그램에서 저장되는 모든 데이터
 - 각 객체는 신원(identity), 타입(class) 값을 가진다.

 * 객체의 신원?
 - 객체가 메모리에 저장된 위치를 가리키는 포인터

 * 객체의 타입?
 - 객체의 내부적인 표현형태와 객체가 지원하는 메서드 및 연산들
 - 특정 타입의 객체가 생성되면 그 객체를 그 타입의 인스턴스(instance)라 부름

 * 객체의 속성와 메서드
 - 속성 : 객체와 연결된 값
 - 메서드 : 호출될 때 객체에 대해 특정연산을 수행하는 함수.

